# Mini-Project: Simulating an (S, s) Inventory Policy

This notebook is designed to grow **together with the course**.  
We will build a simple simulator for a warehouse inventory policy, step by step, as we learn new Python tools.

**Story (informal):**

- A warehouse holds inventory of a product.
- Every day there is some random **demand**.
- If inventory falls below a threshold \(s\), we place an order to raise inventory up to a higher level \(S\).
- We want to simulate this system over many days and see how inventory evolves.

We will build the simulator in four stages:

1. **Session 1:** Think about data types and how to represent the problem.
2. **Session 2:** Write the core daily update loop.
3. **Session 3:** Turn the logic into reusable functions and add randomness.
4. **Session 4:** Save results to a CSV file and make a simple plot.


---
## 1. Session 1 – Understanding the Model (No Code Yet)

Before writing code, let's think about how to **represent** the different pieces of the problem.

We have:

- `inventory` – how many units are in stock at the start of a day (a number).
- `demand` – how many units are requested by customers in a day (a number).
- `S` – the *target* inventory level if we reorder.
- `s` – the *reorder threshold*: if inventory falls below this, we place an order.
- `history` – record of what happens each day (a list).

### Warm-up thinking questions

You can discuss these with a partner or just write your thoughts in text cells:

1. How would you store the inventory level for each day of the simulation?
2. How would you store the daily demand values?
3. What information might be useful if you wanted to analyse the simulation later?


> **Optional exercise:**  
> Create a small example *by hand* on paper or in a text cell:
>
> - Assume initial inventory = 10  
> - For 5 days, choose any demand values you like  
> - Apply the rule “if inventory < s, order up to S” and see how inventory changes.
>
> We will turn this process into code in the next sessions.


---
## 2. Session 2 – Core Daily Loop (Deterministic Version)

Now that we know what we want to simulate, we will write the **core loop** that updates inventory over several days.

To keep it simple for now:

- We will use a **fixed** demand every day (for example, 4 units).
- We ignore costs.
- We ignore randomness.
- We just want to check that our logic for updating inventory is correct.


In [ ]:
# Parameters for the policy
S = 20   # order-up-to level
s = 8    # reorder threshold

# Simulation horizon (number of days)
T = 10

# Initial inventory
inventory = S

# Fixed daily demand for now
daily_demand = 4

print("Day | Inventory at start | Demand | Order | Inventory at end")
print("-" * 60)

for day in range(1, T + 1):
    # TODO: 1) Check if inventory is below s. If so, compute order_qty so that
    # inventory + order_qty becomes S. Otherwise, order_qty = 0.
    order_qty = 0  # <-- replace this with your logic

    # Receive the order immediately (no lead time for now)
    inventory = inventory + order_qty

    # TODO: 2) Apply the demand: compute sales and update inventory
    # For now, assume demand is always fully satisfied (no stockouts).
    demand = daily_demand
    sales = demand  # <-- adjust this if you want to consider stockouts
    inventory = inventory - sales

    print(f"{day:3d} | {inventory + sales:18d} | {demand:6d} | {order_qty:5d} | {inventory:17d}")

> **Task:**  
> Run the cell above, inspect the printed table, and adjust the logic so that:
>
> - An order is placed **only** when inventory is strictly below `s`.
> - The order quantity raises inventory **up to** `S`.
> - Inventory is reduced by the daily demand.
>
> Try different values of `S`, `s`, `T`, and `daily_demand`.


---
## 3. Session 3 – Functions and Random Demand

Now we will:

1. Wrap the daily logic inside a **function** `one_day`.
2. Use another function `simulate` to run many days.
3. Introduce **random demand** using the `random` module.

We model daily demand as a random integer between 0 and 10 units (you can change this later).


In [ ]:
import random

def one_day(inventory, S, s):
    """Simulate one day of operation.

    Parameters
    ----------
    inventory : int or float
        Inventory at the start of the day.
    S : int or float
        Order-up-to level.
    s : int or float
        Reorder threshold.

    Returns
    -------
    new_inventory : int or float
        Inventory at the end of the day.
    demand : int
        Demand realised during the day.
    order_qty : int or float
        Quantity ordered at the start of the day.
    """

    # 1) Generate random demand
    demand = random.randint(0, 10)

    # 2) Decide order quantity based on (S, s) policy
    # TODO: implement the (S, s) rule:
    # If inventory < s, order enough to bring inventory up to S.
    # Otherwise, order 0.
    order_qty = 0  # <-- replace this

    # 3) Receive the order immediately (no lead time)
    inventory = inventory + order_qty

    # 4) Satisfy demand (with possible stockout)
    sales = min(inventory, demand)
    lost_sales = demand - sales
    inventory = inventory - sales

    return inventory, demand, order_qty, lost_sales

In [ ]:
def simulate(T, S, s, initial_inventory=None):
    """Simulate T days of an (S, s) inventory policy.

    Returns a dictionary with time series and summary statistics.
    """
    if initial_inventory is None:
        initial_inventory = S

    inventory = initial_inventory

    history_inventory = []
    history_demand = []
    history_orders = []
    history_lost_sales = []

    for day in range(T):
        inventory, demand, order_qty, lost_sales = one_day(inventory, S, s)
        history_inventory.append(inventory)
        history_demand.append(demand)
        history_orders.append(order_qty)
        history_lost_sales.append(lost_sales)

    results = {
        "inventory": history_inventory,
        "demand": history_demand,
        "orders": history_orders,
        "lost_sales": history_lost_sales,
    }
    return results

In [ ]:
# Try out your simulator

S = 20
s = 8
T = 30

results = simulate(T, S, s)

print("Final inventory:", results["inventory"][-1])
print("Total demand:", sum(results["demand"]))
print("Total orders:", sum(results["orders"]))
print("Total lost sales:", sum(results["lost_sales"]))

> **Tasks:**
>
> 1. Implement the correct (S, s) ordering rule inside `one_day`.
> 2. Run the simulation several times and see how the results change.
> 3. Try different parameter values, for example:
>    - `S = 30`, `s = 10`
>    - `S = 50`, `s = 15`
> 4. Think: how do `S` and `s` affect lost sales and total orders?


### Optional: Adding Simple Costs

We can add two types of costs:

- **Holding cost**: cost per unit of inventory at the *end* of the day (e.g. `h = 0.1` per unit).
- **Penalty cost**: cost per unit of **lost sales** (e.g. `p = 1.0` per unit).

Complete the code below to accumulate these costs.


In [ ]:
def simulate_with_costs(T, S, s, initial_inventory=None, h=0.1, p=1.0):
    if initial_inventory is None:
        initial_inventory = S

    inventory = initial_inventory

    history_inventory = []
    history_demand = []
    history_orders = []
    history_lost_sales = []

    total_holding_cost = 0.0
    total_penalty_cost = 0.0

    for day in range(T):
        inventory, demand, order_qty, lost_sales = one_day(inventory, S, s)

        history_inventory.append(inventory)
        history_demand.append(demand)
        history_orders.append(order_qty)
        history_lost_sales.append(lost_sales)

        # TODO: update costs
        # Hint: holding cost depends on inventory,
        #       penalty cost depends on lost_sales.
        # Example structure:
        # total_holding_cost += h * max(inventory, 0)
        # total_penalty_cost += p * lost_sales

    results = {
        "inventory": history_inventory,
        "demand": history_demand,
        "orders": history_orders,
        "lost_sales": history_lost_sales,
        "holding_cost": total_holding_cost,
        "penalty_cost": total_penalty_cost,
    }
    return results

In [ ]:
# Example: compare two policies

T = 100

policy_A = simulate_with_costs(T, S=20, s=8, h=0.1, p=1.0)
policy_B = simulate_with_costs(T, S=30, s=12, h=0.1, p=1.0)

print("Policy A - holding cost:", policy_A["holding_cost"],
      "penalty cost:", policy_A["penalty_cost"])
print("Policy B - holding cost:", policy_B["holding_cost"],
      "penalty cost:", policy_B["penalty_cost"])

> **Optional tasks:**
>
> 1. Complete the cost calculations in `simulate_with_costs`.
> 2. Compare several policies and see which one gives the lowest total cost
>    (`holding_cost + penalty_cost`).
> 3. How sensitive are the results to the parameters `h` and `p`?


---
## 4. Session 4 – Saving Results and Plotting

Now we will:

1. Save the simulation results to a **CSV file**.
2. Make a simple **plot** of inventory over time.

This will connect what you have learned about file I/O and modules with our mini-project.


In [ ]:
# Run one simulation
S = 20
s = 8
T = 50

results = simulate(T, S, s)

len(results["inventory"]), len(results["demand"])

In [ ]:
# 4.1 Save results to CSV (in the Colab / Jupyter environment)

filename = "inventory_simulation_results.csv"

with open(filename, "w") as f:
    # header
    f.write("day,inventory,demand,order,lost_sales\n")
    # rows
    for day, (inv, dem, order, lost) in enumerate(
        zip(results["inventory"], results["demand"],
            results["orders"], results["lost_sales"]), start=1
    ):
        f.write(f"{day},{inv},{dem},{order},{lost}\n")

print(f"Wrote results to {filename}")

> In Google Colab, you can download the file using:
>
> ```python
> from google.colab import files
> files.download("inventory_simulation_results.csv")
> ```
>
> In a local Jupyter environment, you can simply inspect the file in your working directory.


In [ ]:
# 4.2 Plot inventory over time

import matplotlib.pyplot as plt

plt.plot(results["inventory"])
plt.xlabel("Day")
plt.ylabel("Inventory level")
plt.title("(S, s) inventory policy simulation")
plt.show()

> **Tasks:**
>
> 1. Generate and save a CSV file for at least two different policies.
> 2. Plot the inventory trajectory for each policy (you can reuse the code cell above).
> 3. Visually compare the policies: which one seems more risky (frequent low inventory)?
> 4. (Optional) Plot lost sales over time as well.


---
## 5. Extensions (Optional, For Curious Minds)

If you finish early or want to explore more, here are some ideas:

1. **Lead time**: assume orders arrive after a delay of `L` days instead of immediately.
2. **Multiple products**: simulate two independent products with different (S, s) values.
3. **Different demand distributions**: use `random.gauss` or a custom function for demand.
4. **Service level indicator**: compute the fraction of days with zero lost sales.
5. **Export-and-analyse**: export your CSV and analyse it in the follow-up course using pandas.
